In [2]:
##########################################
#Step 0: Import libraries
##########################################
import os

import pandas as pd

from datetime import date

import matplotlib.pyplot as plt

import numpy as np

import seaborn as sns

import missingno as msno

In [16]:
##########################################
#Step 1: Import data
##########################################

In [17]:
#Setting directory

os.chdir(r'C:\Users\jlenehan\OneDrive - Intel Corporation\Documents\0 - Data Science\Data Analytics Essentials\UCDPA_JohnLenehan')

In [39]:
##PD Incidents DATA
#convert to read_csv for ease - can use chunksize 
#listing json URls and specifying parameters
url_hist = r'https://data.sfgov.org/resource/tmnf-yvry.json?$limit=99999999' #historical incident reports 2003-2018

#instantiating dataframe
sfpd_incident_hist = pd.DataFrame()

#using chunksize method to read in large datasets
sfpd_incident_hist = pd.read_json(url_hist) #reading historical json


In [41]:
#listing json URls and specifying chunksize
url_current = r'https://data.sfgov.org/resource/wg3w-h783.json?$limit=99999999' #current incident reports 2018-present

#instantiating dataframe
sfpd_incident_current = pd.DataFrame()

#using chunksize method to read in large datasets
sfpd_incident_current = pd.read_json(url_current) #reading current incident json


In [42]:
##########################################
#Step 2: Describe data
##########################################

In [43]:
#Describe historical incidents dataset
print(sfpd_incident_hist.columns)
print(sfpd_incident_hist.info())
print(sfpd_incident_hist.describe())
print(sfpd_incident_hist.shape)

print(sfpd_incident_hist.head())

print('\nSFPD Historical Incidents - Unique Values:')
for x in sfpd_incident_hist.columns:
    print(x+':')
    print(sfpd_incident_hist[x].unique())
    print(str(sfpd_incident_hist[x].nunique()) + str(' unique values'))

Index(['pdid', 'incidntnum', 'incident_code', 'category', 'descript',
       'dayofweek', 'date', 'time', 'pddistrict', 'resolution', 'address', 'x',
       'y', 'location', ':@computed_region_26cr_cadq',
       ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv',
       ':@computed_region_6qbp_sg9q', ':@computed_region_qgnn_b9vv',
       ':@computed_region_ajp5_b2md', ':@computed_region_yftq_j783',
       ':@computed_region_p5aj_wyqh', ':@computed_region_fyvs_ahh9',
       ':@computed_region_6pnf_4xz7', ':@computed_region_jwn9_ihcz',
       ':@computed_region_9dfj_4gjx', ':@computed_region_4isq_27mq',
       ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea',
       ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi',
       ':@computed_region_n4xg_c4py', ':@computed_region_fcz8_est8',
       ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129525 entries, 0 to 2129524
Da

[ 4133422003074  5118535807021  4018830907021 ... 17053348465010
 17102776330130 17088142128160]
2129525 unique values
incidntnum:
[ 41334220  51185358  40188309 ... 170004281 170714355 171027763]
1703626 unique values
incident_code:
[ 3074  7021 26030  4134  7020 27195  5041  6244 62020 65016  6362  3021
 16100  6374 64070  6112 30155 28160 65010  6372 12100  6154 64020  6302
 16650 19057  5013  4011 16710 75000  9320  5151 30210  9340 28150  6224
 19050 15150 30191  3014  4145 19055 63010 65015 28100  4154  5053 71010
 27170 15200  7055  5051 19010  4138  6304  6151  3064 64080 68030 71000
  9027 26142 72000  5014  6360 62030  6243  5073  9015 30007 13070 15300
 62010  4013  6221 64085 28165 19015  5011 74000 16020 26170  4063 15095
  9164 26150 61030 16645  7025  6365 68020 62050 16010 19090 73001  7041
  9029  5083 14052 16110  6241 27300 16623 13060 51040  6114  3071  6373
  6363  4021  6303 16664  9330 16621 27010  5072 16620  3424  7046  6314
 30000 73000  9120 30180  5042  4012

847 unique values
dayofweek:
['Monday' 'Tuesday' 'Sunday' 'Friday' 'Thursday' 'Saturday' 'Wednesday']
7 unique values
date:
['2004-11-22T00:00:00.000000000' '2005-10-18T00:00:00.000000000'
 '2004-02-15T00:00:00.000000000' ... '2014-10-18T00:00:00.000000000'
 '2014-10-13T00:00:00.000000000' '2014-10-17T00:00:00.000000000']
5613 unique values
time:
['17:50' '20:00' '02:00' ... '06:19' '05:53' '06:09']
1439 unique values
pddistrict:
['INGLESIDE' 'PARK' 'SOUTHERN' 'TARAVAL' 'TENDERLOIN' 'BAYVIEW' 'CENTRAL'
 'NORTHERN' 'MISSION' 'RICHMOND' 'NA']
11 unique values
resolution:
['NONE' 'ARREST, BOOKED' 'ARREST, CITED' 'PSYCHOPATHIC CASE'
 'NOT PROSECUTED' 'LOCATED' 'UNFOUNDED' 'PROSECUTED BY OUTSIDE AGENCY'
 'DISTRICT ATTORNEY REFUSES TO PROSECUTE'
 'COMPLAINANT REFUSES TO PROSECUTE' 'EXCEPTIONAL CLEARANCE'
 'PROSECUTED FOR LESSER OFFENSE']
12 unique values
address:
['GENEVA AV / SANTOS ST' 'TURK ST / STJOSEPHS AV' 'BRANNAN ST / 1ST ST'
 ... 'LEGIONOFHONOR DR / 34TH AV' 'FRANCISCO ST / LYON ST'

TypeError: unhashable type: 'dict'

In [ ]:
#Describe recent incidents dataset
print(sfpd_incident_current.columns)
print(sfpd_incident_current.info())
print(sfpd_incident_current.describe())
print(sfpd_incident_current.shape)

print(sfpd_incident_current.head())

print('\nSFPD Historical Incidents - Unique Values:')
for x in sfpd_incident_current.columns:
    print(x+':')
    print(sfpd_incident_current[x].unique())
    print(str(sfpd_incident_current[x].nunique()) + str(' unique values'))